# IEMS 351 Lab 
# Solving Constrained Quadratic Programming Problems via cyipopt

IPOPT, short for "Interior Point OPTimizer, pronounced I-P-Opt", is a software library for large scale nonlinear optimization of continuous systems. 
IPOPT was originally developed by Andreas Wächter and Lorenz T. Biegler of the Department of Chemical Engineering at Carnegie Mellon University. Their work was recognized with the INFORMS Computing Society Prize in 2009.

(Reference: https://en.wikipedia.org/wiki/IPOPT#:~:text=IPOPT%2C%20short%20for%20%22Interior%20Point,August%2026%2C%202005)

cyipopt is a Python wrapper for the Ipopt optimization package, written in Cython.

(Reference: https://cyipopt.readthedocs.io/en/stable/)


In this lab, please creae a new conda environment because there may be some dependency issues while installing cyipopt. 
## Create a new conda environemnt
conda create --name <env_name>
## Install cyipopt
conda install -c conda-forge cyipopt
## Install other necessary packages 
pip install matplotlib

pip install jupyterlab

pip install scipy

References: https://cyipopt.readthedocs.io/en/stable/tutorial.html

## Exercise 1: Test on cyipopt 

In [ ]:
import cyipopt
import numpy as np
from cyipopt import minimize_ipopt
from scipy.optimize import rosen, rosen_der

In [ ]:
x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
res = minimize_ipopt(rosen, x0, jac=rosen_der)
print(res)

## Exercise 2: Quadratic Programming Problems with Linear Constraints  
$$
\begin{aligned}
\min \ & d + g^\top x + x^\top Q x \\
\text{s.t.} \ C_L & \leq A x \leq C_U \\
& x_L \leq x \leq x_U
\end{aligned}
$$

In [ ]:
class QP_solver(cyipopt.Problem):
    def set_problem_param(self, d, g, Q, A):
        self._d = d
        self._g = g
        self._Q = Q
        self._A = A
        self._dim = len(g)
        self._jacobian = np.concatenate(A)
        self._hessian = 2 * Q
        self._hessianstructure = np.nonzero(np.tril(np.ones((self._dim, self._dim))))

    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        return self._d + self._g @ x + x @ self._Q @ x

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return self._g + 2 * self._Q @ x

    def constraints(self, x):
        """Returns the constraints."""

        return self._A @ x

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        return self._jacobian

    # def hessianstructure(self):
    #    """Returns the row and column indices for non-zero vales of the
    #    Hessian."""

    # NOTE: The default hessian structure is of a lower triangular matrix,
    # therefore this function is redundant. It is included as an example
    # for structure callback.

    #    return np.nonzero(np.tril(np.ones((self._dim , self._dim))))

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""

        row, col = self._hessianstructure

        return obj_factor * self._hessian[row, col]

    def intermediate(
        self,
        alg_mod,
        iter_count,
        obj_value,
        inf_pr,
        inf_du,
        mu,
        d_norm,
        regularization_size,
        alpha_du,
        alpha_pr,
        ls_trials,
    ):
        """Prints information at every Ipopt iteration."""

        msg = "Objective value at iteration #{:d} is - {:g}"

        print(msg.format(iter_count, obj_value))

## Projection Problem 
$$
\min_{x \in X} \frac{1}{2} \|x - y \|^2
$$


## Problem 1 
$$
\begin{aligned}
\min \ & 0.5 x_1^2 + 0.5 x_2^2 \\
\text{s.t.} \ & -x_1 - x_2 \leq -1
\end{aligned}
$$

It is equivalent to project the origin $(0,0)$ onto the convex set $X = \{x \in \mathbb{R}^2: x_1 + x_2 \geq 1 \}$. 

In [ ]:
lb = [-2.0e19, -2.0e19]
ub = [2.0e19, 2.0e19]
cl = [-2.0e19]
cu = [-1]
d = 0
g = np.array([0, 0])
Q = np.array([[0.5, 0], [0, 0.5]])
A = np.array([[-1, -1]])

x0 = [1, 0]

nlp_qp_solver = QP_solver(
    n=len(x0),
    m=len(cl),
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu,
)
nlp_qp_solver.set_problem_param(d, g, Q, A)

In [ ]:
x, info = nlp_qp_solver.solve(x0)

In [ ]:
print("The optimal solution of Problem 1:")
print(x)

## Problem 2
Projecting a point $y$ onto $X = \{x \in \mathbb{R}^2: -x_1 + x_2 \leq 1, \ x_1 - x_2 \leq 1, \ 0 \leq x_1 \leq 2, \ 0 \leq x_2 \leq 3 \}$ can be formulated as the following problem: 
$$
\begin{aligned}
\min \ & 0.5 x_1^2 + 0.5 x_2^2 - y_1 x_1 - y_2 x_2 \\
\text{s.t.} \ & -x_1 + x_2 \leq 1 \\
&  x_1 - x_2 \leq 1 \\
& 0 \leq x_1 \leq 2 \\
& 0 \leq x_2 \leq 3.
\end{aligned}
$$

In [ ]:
# given y = (0,2)
y = np.array([0, 2])

lb = [0, 0]
ub = [2, 3]
cl = [-2.0e19, -2.0e19]
cu = [1, 1]
d = 0
g = -y
Q = np.array([[0.5, 0], [0, 0.5]])
A = np.array([[-1, 1], [1, -1]])

nlp_qp2_solver = QP_solver(
    n=len(x0),
    m=len(cl),
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu,
)
nlp_qp2_solver.set_problem_param(d, g, Q, A)

In [ ]:
x, info = nlp_qp2_solver.solve(x0)

In [ ]:
print("The optimal solution of Problem 2:")
print(x)

## Problem 3 
$$
\begin{aligned}
    \min \ & 2 x_1^2 + x_2^2 + x_1 x_2 + x_1 + x_2 \\
    \text{s.t.} \ & x_1 + x_2 = 1 \\
                  & x_1  \geq 0, \ x_2 \geq 0
\end{aligned}
$$

In [ ]:
lb = [0, 0]
ub = [1e10, 1e10]
cl = [1]
cu = [1]
d = 0
g = np.array([1, 1])
Q = np.array([[2, 0.5], [0.5, 1]])
A = np.array([[1, 1]])

nlp_qp2_solver = QP_solver(
    n=len(x0),
    m=len(cl),
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu,
)
nlp_qp2_solver.set_problem_param(d, g, Q, A)

In [ ]:
x0 = [0, 0]
x, info = nlp_qp2_solver.solve(x0)

In [ ]:
print("The optimal solution of Problem 2:")
print(x)

# Exercise 3: A Constrained Nonlinear Optimization Problem 
$$
\begin{aligned}
\min_{x \in \mathbb{R}^4} \ &x_1 x_4 (x_1 + x_2 + x_3) + x_3 \\
\text{s.t.} \ & x_1 x_2 x_3 x_4 \geq 25 \\
& x_1^2 + x_2^2 + x_3^2 + x_4^2 = 40 \\
&1 \leq x_1, x_2, x_3, x_4 \leq 5.
\end{aligned}
$$

The optimal solution is $x^* = (1.0, 4.743, 3.821, 1.379)$

In [ ]:
class HS071(cyipopt.Problem):

    def objective(self, x):
        """Returns the scalar value of the objective given x."""
        return x[0] * x[3] * np.sum(x[0:3]) + x[2]

    def gradient(self, x):
        """Returns the gradient of the objective with respect to x."""
        return np.array(
            [
                x[0] * x[3] + x[3] * np.sum(x[0:3]),
                x[0] * x[3],
                x[0] * x[3] + 1.0,
                x[0] * np.sum(x[0:3]),
            ]
        )

    def constraints(self, x):
        """Returns the constraints."""
        return np.array((np.prod(x), np.dot(x, x)))

    def jacobian(self, x):
        """Returns the Jacobian of the constraints with respect to x."""
        return np.concatenate((np.prod(x) / x, 2 * x))

    def hessianstructure(self):
        """Returns the row and column indices for non-zero vales of the
        Hessian."""

        # NOTE: The default hessian structure is of a lower triangular matrix,
        # therefore this function is redundant. It is included as an example
        # for structure callback.

        return np.nonzero(np.tril(np.ones((4, 4))))

    def hessian(self, x, lagrange, obj_factor):
        """Returns the non-zero values of the Hessian."""

        H = obj_factor * np.array(
            (
                (2 * x[3], 0, 0, 0),
                (x[3], 0, 0, 0),
                (x[3], 0, 0, 0),
                (2 * x[0] + x[1] + x[2], x[0], x[0], 0),
            )
        )

        H += lagrange[0] * np.array(
            (
                (0, 0, 0, 0),
                (x[2] * x[3], 0, 0, 0),
                (x[1] * x[3], x[0] * x[3], 0, 0),
                (x[1] * x[2], x[0] * x[2], x[0] * x[1], 0),
            )
        )

        H += lagrange[1] * 2 * np.eye(4)

        row, col = self.hessianstructure()

        return H[row, col]

    def intermediate(
        self,
        alg_mod,
        iter_count,
        obj_value,
        inf_pr,
        inf_du,
        mu,
        d_norm,
        regularization_size,
        alpha_du,
        alpha_pr,
        ls_trials,
    ):
        """Prints information at every Ipopt iteration."""
        iterate = self.get_current_iterate()
        infeas = self.get_current_violations()
        primal = iterate["x"]
        jac = self.jacobian(primal)

        print("Iteration:", iter_count)
        print("Primal iterate:", primal)
        print("Flattened Jacobian:", jac)
        print("Dual infeasibility:", infeas["grad_lag_x"])

In [ ]:
lb = [1.0, 1.0, 1.0, 1.0]
ub = [5.0, 5.0, 5.0, 5.0]

cl = [25.0, 40.0]
cu = [2.0e19, 40.0]

x0 = [1.0, 5.0, 5.0, 1.0]

nlp = HS071(
    n=len(x0),
    m=len(cl),
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu,
)

x, info = nlp.solve(x0)

In [ ]:
print(x)

## Further Readings 
https://cyipopt.readthedocs.io/en/stable/tutorial.html

https://cyipopt.readthedocs.io/en/stable/reference.html#reference

https://link.springer.com/article/10.1007/S10107-004-0559-Y